# basic genetic algorithm for music

make sure to pip install the following before attempting this ex: <br>
pyaudio, numpy, midiutil

## helper music functions

In [1]:
def note_frequency(note, octave): # returns the frequency associated with a given note in its numerical form
    A4_freq = 440
    A4_note_number = 57
    note_number = note_to_number(note, octave)
    return A4_freq * 2 ** ((note_number - A4_note_number) / 12)

def note_to_number(note, octave): # converts a note to its corresponding number
    note_map = {"C": 0, "C#": 1, "Db": 1, "D": 2, "D#": 3, "Eb": 3, "E": 4, "F": 5, "F#": 6, "Gb": 6, "G": 7, "G#": 8, "Ab": 8, "A": 9, "A#": 10, "Bb": 10, "B": 11}
    if type(note) == str: # if we are given a note as a string like "C"
        return 12 * (octave + 1) + note_map[note]
    else: # if we are given the note as a number 
        return 12 * (octave + 1) + note

def sine_wave(frequency, length, rate=44100): # given a frequency and length, returns a sin wave, (rate is the sampling rate detail of sound)
    t = np.linspace(0, length, int(rate * length))
    wave = np.sin(frequency * 2 * np.pi * t)
    return wave

def cos_wave(frequency, length, rate=44100): # given a frequency and length, returns a cos wave 
    t = np.linspace(0, length, int(rate * length))
    wave = np.cos(np.pi/4 + frequency * 2 * np.pi * t)
    return wave

def generate_white_noise(rate=44100, length=0.05, volume=0.5): # generate uniform random noise given a rate, length and volume
    return np.random.uniform(-volume, volume, int(rate * length))

def generate_sharp_drum_sound(rate=44100, length=0.1, sine_freq=150, volume=0.5): # to generate the metronome tick sound
    """Generate a sharper drum sound."""
    t = np.linspace(0, length, int(rate * length))
    
    # Sine wave for body of the drum sound
    sine_wave = np.sin(2 * np.pi * sine_freq * t) * volume

    # Quick decay envelope for sharpness
    decay = np.e ** (-15 * t)

    # Add a bit of noise for sharpness
    noise = np.random.uniform(-0.5, 0.5, sine_wave.shape) * decay

    # Combine sine wave, noise, and decay
    drum_sound = (sine_wave + noise) * decay

    return drum_sound

def play_sound(sound, rate=44100, fade_out_duration=1):  # Fade out in the last 1 second
    # Calculate the number of samples for the fade out
    fade_out_samples = int(rate * fade_out_duration)

    # Extend the sound by repeating the last value for the fade-out duration
    extended_sound = np.append(sound, np.full(fade_out_samples, sound[-1]))

    # Create a fade-out envelope (exponential decay)
    fade_out_envelope = np.linspace(1, 0, fade_out_samples)**2

    # Apply the fade-out envelope to the extended part of the sound
    extended_sound[-fade_out_samples:] *= fade_out_envelope

    # Play the sound with fade-out
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paFloat32, channels=1, rate=rate, output=True)
    stream.write(extended_sound.astype(np.float32).tobytes())
    stream.stop_stream()
    stream.close()
    p.terminate()


## code to produce music and metronome tick in the background

In [2]:
import numpy as np
import pyaudio

# ... (Include all the previously defined functions like note_frequency, sine_wave, etc.)

def play_notes_and_chords_with_metronome(chords, octaves, lengths, metronome_tick_length=0.05, ticks_per_beat=4):
    """Play a list of chords with corresponding octaves and lengths, along with a metronome."""
    if not (len(chords) == len(octaves) == len(lengths)):
        raise ValueError("The length of chords, octaves, and lengths lists must be the same.")

    # Generate notes stream
    notes_stream = np.array([], dtype=np.float32)
    total_length = 0  # To calculate total length for the metronome stream
    for chord, octave, length in zip(chords, octaves, lengths):
        total_length += length
        if chord:  # If not an empty list (silence)
            chord_wave = np.zeros(int(length * 44100))  # Start with silence
            for note in chord:  # For each note in the chord
                frequency = note_frequency(note, octave)
                note_wave = sine_wave(frequency, length)
                chord_wave += note_wave
            chord_wave /= len(chord)  # Normalize volume of chord
            notes_stream = np.concatenate((notes_stream, chord_wave))
        else:  # For a blank note (silence)
            silence = np.zeros(int(length * 44100))
            notes_stream = np.concatenate((notes_stream, silence))

    # Generate metronome stream
    metronome_stream = np.array([], dtype=np.float32)
    total_metronome_ticks = int(total_length * ticks_per_beat / metronome_tick_length)
    for _ in range(total_metronome_ticks):
        metronome_stream = np.concatenate((metronome_stream, generate_sharp_drum_sound(length=metronome_tick_length)))
        metronome_stream = np.concatenate((metronome_stream, np.zeros(int((1 / ticks_per_beat - metronome_tick_length) * 44100))))

    # Ensure metronome stream is not longer than notes stream
    if len(metronome_stream) > len(notes_stream):
        metronome_stream = metronome_stream[:len(notes_stream)]

    # Mix the metronome and notes streams
    mixed_stream = metronome_stream + notes_stream

    # Play the mixed audio
    play_sound(mixed_stream)



### example of its usage

In [3]:
# Example usage note that the bottom 3 lists must have the same length
chords_list = [['C', 'E', 'G'], ['D', 'F', 'A'], [], ['D', 'F', 'A'],['C', 'E', 'G'],['D', 'F', 'A'],['C', 'E', 'G']] # the chords to play
octaves_list = [3, 3, 3, 4, 4,4,4] # the octaves of each chord
lengths_list = [0.5, 0.25, 0.75, 0.25, 1, 0.2,0.25] # the lengths that each chord are sustained for
play_notes_and_chords_with_metronome(chords_list, octaves_list, lengths_list)
#play_notes_and_chords_with_metronome(chords_list, octaves_list, lengths_list)


### genetic algorithm framework, we coded this in ex1
<br> feel free to paste in the one you coded

In [6]:
from random import choices, randint, randrange, random, sample
from typing import List, Optional, Callable, Tuple

Genome = List[int]
Population = List[Genome]
PopulateFunc = Callable[[], Population]
FitnessFunc = Callable[[Genome], int]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]
PrinterFunc = Callable[[Population, int, FitnessFunc], None]


def generate_genome(length: int) -> Genome:
    return choices([0, 1], k=length)


def generate_population(size: int, genome_length: int) -> Population:
    return [generate_genome(genome_length) for _ in range(size)]


def single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    if len(a) != len(b):
        raise ValueError("Genomes a and b must be of same length")

    length = len(a)
    if length < 2:
        return a, b

    p = randint(1, length - 1)
    return a[0:p] + b[p:], b[0:p] + a[p:]


def mutation(genome: Genome, num: int = 1, probability: float = 0.5) -> Genome:
    for _ in range(num):
        index = randrange(len(genome))
        genome[index] = genome[index] if random() > probability else abs(genome[index] - 1)
    return genome


def population_fitness(population: Population, fitness_func: FitnessFunc) -> int:
    return sum([fitness_func(genome) for genome in population])


def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return sample(
        population=generate_weighted_distribution(population, fitness_func),
        k=2
    )


def generate_weighted_distribution(population: Population, fitness_func: FitnessFunc) -> Population:
    result = []

    for gene in population:
        result += [gene] * int(fitness_func(gene)+1)

    return result


def sort_population(population: Population, fitness_func: FitnessFunc) -> Population:
    return sorted(population, key=fitness_func, reverse=True)


def genome_to_string(genome: Genome) -> str:
    return "".join(map(str, genome))


def print_stats(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("GENERATION %02d" % generation_id)
    print("=============")
    print("Population: [%s]" % ", ".join([genome_to_string(gene) for gene in population]))
    print("Avg. Fitness: %f" % (population_fitness(population, fitness_func) / len(population)))
    sorted_population = sort_population(population, fitness_func)
    print(
        "Best: %s (%f)" % (genome_to_string(sorted_population[0]), fitness_func(sorted_population[0])))
    print("Worst: %s (%f)" % (genome_to_string(sorted_population[-1]),
                              fitness_func(sorted_population[-1])))
    print("")

    return sorted_population[0]


def run_evolution(
        populate_func: PopulateFunc,
        fitness_func: FitnessFunc,
        fitness_limit: int,
        selection_func: SelectionFunc = selection_pair,
        crossover_func: CrossoverFunc = single_point_crossover,
        mutation_func: MutationFunc = mutation,
        generation_limit: int = 100,
        printer: Optional[PrinterFunc] = None) \
        -> Tuple[Population, int]:
    population = populate_func()

    i = 0
    for i in range(generation_limit):
        population = sorted(population, key=lambda genome: fitness_func(genome), reverse=True)

        if printer is not None:
            printer(population, i, fitness_func)

        if fitness_func(population[0]) >= fitness_limit:
            print("reached fitness limit")
            break

        next_generation = population[0:2] # transfer the two fittest beings
        for j in range(int((len(population)) / 2)-1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

    return population, i


# Writing code to generate melodies with notes of different lengths and pauses.

In [ ]:
from midiutil import MIDIFile

saver = {}
number_map = {0 : "C", 1: "C#", 1: "Db", 2: "D", 3: "D#", 3: "Eb", 4: "E", 5: "F", 6: "F#", 6: "Gb", 7:"G", 8: "G#", 8: "Ab", 9: "A", 10: "A#",10: "Bb", 11: "B"}

def bin_to_dec(num: list):
    out = 0
    for i in range(len(num)):
        out += (2 ** i) * num[i]
    return out

def genome_to_num_list(genome: Genome): # returns the length list and the play_notes list as numbers
    assert len(genome) % 6 == 0
    play_notes = []
    lengths = []
    # TODO convert our genome into a notes list and a lengths list, interpreting every 6 set of 0s and 1s as one note:
    # the first 4 bits as the note, and the next 2 as the length.
    # Note: since there are 16 binary numbers of length 4, you are free to decide what you want to do with the extras
    # Note: there should be a empty note, which would sound like pauses in the music
    # return the play_notes in their numerical form

    return play_notes, lengths

def genome_to_list(genome: Genome): # returns the length list and the play_notes list as notes
    assert len(genome) % 6 == 0
    play_notes, lengths = genome_to_num_list(genome)
    # TODO convert play_notes to letters instead of numbers using the number_map dictionary
    # Note: play_notes is a list of lists
    # See how to use a dictionary in the IntrodudingPythonSyntax.ipynb.

    return play_notes, lengths
    

def genome_to_music(genome: Genome): # encoding: (2 bits for length) (4 bits for notes)
    # TODO play the music that is encoded by a genome
    # hint: use the genome_to_list method and the play_notes_and_chords_with_metronome method
    # Note: we havent implemented octaves yet so set the octave 4 for every note
    pass

def fitness_music(genome: Genome):
    num = bin_to_dec(genome)
    if num in saver:
        return saver[num]
    genome_to_music(genome)
    rating = int(input("rate this song from 1-5"))
    if int(rating) == 100:
        save_melody(genome)
    saver[bin_to_dec(genome)] = int(rating)
    print(int(rating))
    return int(rating)

def save_melody(genome: Genome):
    track = 0
    channel = 0
    time = 0  # In beats
    duration = 1  # In beats
    tempo = 120  # In BPM
    volume = 100  # 0-127, as per the MIDI standard

    MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created)
    MyMIDI.addTempo(track, time, tempo)
    
    play_notes, lengths = genome_to_num_list(genome)
    timer = 0
    for i in range(len(lengths)):
        for j in play_notes[i]:
            MyMIDI.addNote(track, channel, 33 + j,timer, lengths[i], volume)
        timer += lengths[i]
    with open("GeneticAlgoMusic.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file)
        
        
def print_func(population: Population, generation_id: int, fitness_func: FitnessFunc):
    print("NEXT GEN")


run_evolution(lambda : generate_population(5,60), fitness_music, 10, selection_pair, single_point_crossover, mutation, 10, print_func)
